In [1]:
from tqdm import tqdm
import json

from videograph import VideoGraph
from utils.general import *
from utils.video_processing import *
from utils.chat_api import *
from prompts import *

from retrieve import answer_with_retrieval
from generate_memory import streaming_process_video

processing_config = json.load(open("configs/processing_config.json"))
memory_config = json.load(open("configs/memory_config.json"))

/usr/local/lib/python3.9/dist-packages/bytedmetrics/__init__.py:10: UserWarning: bytedmetrics is renamed to bytedance.metrics, please using `bytedance.metrics` instead of `bytedmetrics`
  warnings.warn("bytedmetrics is renamed to bytedance.metrics, please using `bytedance.metrics` instead of `bytedmetrics`")
/usr/local/lib/python3.9/dist-packages/bytedtrace/__init__.py:108: UserWarning: [bytedtrace] global tracer is already initialized.
  warnings.warn('[bytedtrace] global tracer is already initialized.')


In [ ]:
# video paths can be paths to directories or paths to mp4 files
video_paths = ["/mnt/hdfs/foundation/longlin.kylin/mmagent/data/test_video_clips/test2"]
save_dir = processing_config["save_dir"]
max_workers = 1


def process_single_video(video_path):
    video_graph = VideoGraph(**memory_config)
    streaming_process_video(video_graph, video_path)


# Process videos in parallel using ThreadPoolExecutor with max_workers limit
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Process videos in parallel using map
    list(
        tqdm(
            executor.map(process_single_video, video_paths),
            total=len(video_paths),
            desc="Processing videos",
        )
    )

In [ ]:
video_graph_path = (
    "data/mems/5-Poor-People-vs-1-Secret-Millionaire_60_5_-1_10_20_0.3_0.6.pkl"
)
video_graph = load_video_graph(video_graph_path)

question = "How might the social development of children living in military families be affected?"

answer = answer_with_retrieval(
    video_graph,
    question,
    query_num=processing_config["query_num"],
    topk=processing_config["topk"],
    mode="argmax",
)

In [ ]:
video_graph_path = "/mnt/hdfs/foundation/longlin.kylin/mmagent/data/mems/CZ_1/-UhacNNM_HU_30_5_-1_10_20_0.3_0.6.pkl"
video_graph = load_video_graph(video_graph_path)
video_graph.visualize()

# print all episodic and semantic nodes
for node_id in video_graph.text_nodes:
    print(video_graph.nodes[node_id].metadata["contents"][0])

In [ ]:
import numpy as np
from tqdm import tqdm
import os
import json
from concurrent.futures import ProcessPoolExecutor
import multiprocessing

from videograph import VideoGraph
from utils.general import *
from utils.video_processing import *
from utils.chat_api import *
from prompts import *

from face_processing import process_faces
from voice_processing import process_voices
from memory_processing import (
    process_captions,
    generate_captions_and_thinkings_with_ids,
)
from generate_memory import streaming_process_video

processing_config = json.load(open("configs/processing_config.json"))
memory_config = json.load(open("configs/memory_config.json"))

video_paths = [
    "/mnt/hdfs/foundation/longlin.kylin/mmagent/data/video_clips/PnvZZwlN2yk"
]

save_dir = processing_config["save_dir"]
generated_memories = os.listdir(save_dir)
generated_memories = [
    generated_memory
    for generated_memory in generated_memories
    if generated_memory.endswith(".pkl")
]
video_paths = [
    video_path
    for video_path in video_paths
    if generate_file_name(video_path) + ".pkl" not in generated_memories
]

# save_dir = processing_config["save_dir"]
# video_paths = ['/mnt/hdfs/foundation/longlin.kylin/mmagent/data/video_clips/EodRBU-HVEI']

cpu_count = multiprocessing.cpu_count()
max_workers = min(cpu_count, processing_config.get("max_parallel_videos", 4))

print(f"Using {max_workers} processes (CPU cores: {cpu_count})")

preprocessing = []


def process_single_video(video_path):
    video_graph = VideoGraph(**memory_config)
    try:
        streaming_process_video(video_graph, video_path, preprocessing=preprocessing)
    except Exception as e:
        log_dir = processing_config["log_dir"]
        os.makedirs(log_dir, exist_ok=True)
        with open(os.path.join(log_dir, f"generate_memory_error.log"), "a") as f:
            f.write(f"Error processing video {video_path}: {e}\n")


with ProcessPoolExecutor(max_workers=max_workers) as executor:
    list(
        tqdm(
            executor.map(process_single_video, video_paths),
            total=len(video_paths),
            desc="Processing videos",
        )
    )

In [1]:
from utils.general import load_video_graph
from retrieve import answer_with_retrieval

video_graph_path = "/mnt/hdfs/foundation/longlin.kylin/mmagent/data/mems/CZ_1/-UhacNNM_HU_30_5_-1_10_20_0.3_0.6.pkl"
video_graph = load_video_graph(video_graph_path)

question = "How many dance performances are there in total in the video?"
answer = answer_with_retrieval(video_graph, question)

Loading video graph from /mnt/hdfs/foundation/longlin.kylin/mmagent/data/mems/CZ_1/-UhacNNM_HU_30_5_-1_10_20_0.3_0.6.pkl
Generating action 0 times
The provided knowledge does not contain any information about the number of dance performances in the video. To answer this question, I need to retrieve information about the occurrences of dance performances, including their frequency and any specific moments or clips where they are mentioned or shown. Therefore, I will generate queries to retrieve this missing information.

[SEARCH]  
[  
	"Total number of dance performances in the video.",  
	"Descriptions of dance performances in the video.",  
	"Clips or moments featuring dance performances.",  
	"CLIP_1",  
	"CLIP_2",  
	"CLIP_3"  
]  
Total number of dance performances in the video.
Descriptions of dance performances in the video.
Clips or moments featuring dance performances.
Queries: ['Total number of dance performances in the video.', 'Descriptions of dance performances in the vide

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2352678/2393581176.py", line 8, in <module>
    answer = answer_with_retrieval(video_graph, question)
  File "/mnt/bn/videonasi18n/longlin.kylin/mmagent/retrieve.py", line 338, in answer_with_retrieval
UnboundLocalError: local variable 'final_answer' referenced before assignment

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 2144, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/ultratb.py", line 1435, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/ultratb.py", line 1326, 